In [7]:
# ============================================================
# AASG + SPRV  (LIGHTWEIGHT / STABLE / JP UI)
# - Pro mode: direct TID specification per step
# - Multi-tactic per step
# - Asset selection per step (1 asset) with back/restart
# - Detection "Nested Practice Mode" (apply ALL at once)
# - Technique_ID normalization (fix "候補に出てるのに見つからない" 問題)
# - Tree cap auto-control (prevents freeze)
# ============================================================

import pandas as pd
import numpy as np
import itertools
from pathlib import Path
import re
import unicodedata

# =============================
# Default Paths (Colab)
# =============================
DEFAULT_FREQ_CSV = "/mnt/data/ARKS_Task1_with_Pi_and_DataSources.csv"
DEFAULT_DICT_CSV = "/mnt/data/arks_dictionary_J_ver3_phase0_asset_independent.csv"

DEFAULT_LG_ASSET_TXT = "/mnt/data/LG Asset.txt"
DEFAULT_BOE_ASSET_TXT = "/mnt/data/BoE Asset.txt"
DEFAULT_HEALTH_ASSET_TXT = "/mnt/data/Health Asset.txt"

# =============================
# Helpers
# =============================
def _clean_input_path(s: str, default: str) -> str:
    s = (s or "").strip()
    if not s:
        return default
    s = re.sub(r"^\s*default\s*:\s*", "", s, flags=re.IGNORECASE).strip()
    return s

def ask_yn_jp(prompt: str, default_yes: bool = True) -> bool:
    d = "y" if default_yes else "n"
    while True:
        ans = input(f"{prompt} [y/n] (default {d}): ").strip().lower()
        if ans == "":
            return default_yes
        if ans in ("y", "yes"):
            return True
        if ans in ("n", "no"):
            return False
        print("入力が不正です。y または n を入力してください。")

def ask_choice_jp(prompt: str, choices: list[str], default: str | None = None) -> str:
    choices_l = [c.lower() for c in choices]
    default_disp = f"(default {default})" if default else ""
    while True:
        ans = input(f"{prompt} {default_disp}: ").strip().lower()
        if ans == "" and default:
            ans = default.lower()
        if ans in choices_l:
            return choices[choices_l.index(ans)]
        print(f"入力が不正です。選択肢: {choices}")

def ask_int(prompt: str, min_v: int, max_v: int, default: int | None = None) -> int:
    d = f"(default {default})" if default is not None else ""
    while True:
        s = input(f"{prompt} [{min_v}-{max_v}] {d}: ").strip()
        if s == "" and default is not None:
            return default
        if s.isdigit():
            v = int(s)
            if min_v <= v <= max_v:
                return v
        print(f"入力が不正です。{min_v}〜{max_v} の整数を入力してください。")

def pick_first_existing_col(df: pd.DataFrame, candidates: list[str]) -> str | None:
    for c in candidates:
        if c in df.columns:
            return c
    return None

def load_asset_list_from_txt(path: str) -> list[str]:
    p = Path(path)
    if not p.exists():
        return []
    lines = [x.strip() for x in p.read_text(encoding="utf-8").splitlines()]
    return [x for x in lines if x and not x.startswith("#")]

# =============================
# Technique_ID Normalization  ★重要
# =============================
def normalize_tid(x: str) -> str:
    """
    Normalize Technique ID:
    - NFKC normalize (full-width -> half-width)
    - trim spaces
    - allow "1078.004" -> "T1078.004"
    - ensure subtech padded to 3 digits: T1078.4 -> T1078.004
    """
    if x is None:
        return ""
    s = str(x)
    s = unicodedata.normalize("NFKC", s)
    s = s.strip()
    s = s.replace(" ", "")
    s = s.upper()

    # remove accidental quotes/commas
    s = s.strip(",").strip()

    if s == "":
        return ""

    # Accept forms like "T1078.004" or "1078.004" or "T1078"
    if not s.startswith("T"):
        # if looks like digits or digits.subdigits
        if re.match(r"^\d{4}(\.\d+)?$", s):
            s = "T" + s

    m = re.match(r"^(T\d{4})(?:\.(\d+))?$", s)
    if not m:
        return s  # unknown format; return as-is

    parent = m.group(1)
    sub = m.group(2)
    if sub is None:
        return parent
    sub = sub.zfill(3)  # pad
    return f"{parent}.{sub}"

def parse_tid_list(user_input: str) -> list[str]:
    """
    Accept: "T1566.001 T1190" or "T1566.001,T1190" or mix
    """
    s = (user_input or "").strip()
    if not s:
        return []
    # split by comma or whitespace
    parts = re.split(r"[,\s]+", s.strip())
    parts = [p for p in parts if p]
    return [normalize_tid(p) for p in parts if normalize_tid(p)]

# =============================
# Dictionary (TID -> Japanese name)
# =============================
def load_dictionary(dict_path: str) -> dict[str, str]:
    try:
        df = pd.read_csv(dict_path)
    except Exception:
        return {}
    tid_col = pick_first_existing_col(df, ["Technique_ID", "TechniqueID", "TID", "technique_id"])
    name_col = pick_first_existing_col(df, ["Technique_Name_J", "Technique_Name_JP", "TechniqueName_J", "Technique_Name", "Name"])
    if tid_col is None or name_col is None:
        return {}
    df[tid_col] = df[tid_col].astype(str).map(normalize_tid)
    df[name_col] = df[name_col].astype(str)
    return dict(zip(df[tid_col], df[name_col]))

# =============================
# Load Frequency
# =============================
def load_frequency(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)

    # Normalize Technique_ID
    tid_col = pick_first_existing_col(df, ["Technique_ID", "TechniqueID", "TID"])
    if tid_col is None:
        raise ValueError(f"Frequency CSVにTechnique_ID相当列がありません。Columns={df.columns.tolist()}")
    if tid_col != "Technique_ID":
        df = df.rename(columns={tid_col: "Technique_ID"})
    df["Technique_ID"] = df["Technique_ID"].astype(str).map(normalize_tid)

    # Normalize Tactic_ID + explode
    tac_col = pick_first_existing_col(df, ["Tactic_ID", "TacticID", "tactic_id", "Tactics"])
    if tac_col is None:
        raise ValueError(f"Frequency CSVにTactic_ID相当列がありません。Columns={df.columns.tolist()}")
    if tac_col != "Tactic_ID":
        df = df.rename(columns={tac_col: "Tactic_ID"})
    df["Tactic_ID"] = df["Tactic_ID"].astype(str).apply(lambda s: unicodedata.normalize("NFKC", s).strip())
    # split by comma/semicolon/pipe
    df["Tactic_ID"] = df["Tactic_ID"].str.split(r"[;,|,]")
    df = df.explode("Tactic_ID")
    df["Tactic_ID"] = df["Tactic_ID"].astype(str).str.strip()
    df = df[df["Tactic_ID"] != ""]

    # Flags (optional)
    if "No_Mitigation_Flag" not in df.columns:
        df["No_Mitigation_Flag"] = 0
    if "No_PCD_Flag" not in df.columns:
        # 1 means "NOT P/C/D" in your earlier convention
        df["No_PCD_Flag"] = 1

    # Pi
    if "Pi" not in df.columns:
        bin5_col = pick_first_existing_col(df, ["Freq_Total_Pct_BIN5", "Freq_Total_Pct_BIN", "FreqBin5", "BIN5"])
        if bin5_col is None:
            raise ValueError("Pi列が無く、Freq_Total_Pct_BIN5等も見つかりません。")
        df["Pi"] = pd.to_numeric(df[bin5_col], errors="coerce").fillna(0.0) * 0.2 * 0.9
    df["Pi"] = pd.to_numeric(df["Pi"], errors="coerce").fillna(0.0)

    # Detection list and DS_Total
    if "Detection_DataSources_List" not in df.columns:
        df["Detection_DataSources_List"] = ""
    df["Detection_DataSources_List"] = df["Detection_DataSources_List"].fillna("").astype(str)

    if "DS_Total" not in df.columns:
        # if count column exists, map it
        ds_cnt_col = pick_first_existing_col(df, ["Detection_DataSources_Count", "Detection_DataSources_Count ", "DS_total"])
        if ds_cnt_col is not None:
            df["DS_Total"] = pd.to_numeric(df[ds_cnt_col], errors="coerce")
        else:
            # count from list
            df["DS_Total"] = df["Detection_DataSources_List"].apply(
                lambda x: len([t for t in str(x).split(";") if t.strip()])
            )
    df["DS_Total"] = pd.to_numeric(df["DS_Total"], errors="coerce").fillna(0).astype(int)

    # Sector_Add (optional)
    if "Sector_Add" not in df.columns:
        df["Sector_Add"] = 0
    df["Sector_Add"] = pd.to_numeric(df["Sector_Add"], errors="coerce").fillna(0).astype(int)

    return df

# =============================
# Mi computation (avoid 0/1)
# =============================
def mi_from_maturity(maturity: float) -> float:
    """
    Base: Mi = 1 - (maturity * 0.9)
    Clamp to [0.2, 0.9] to avoid 0/1 per your design intent.
    """
    if maturity is None or np.isnan(maturity):
        maturity = 0.0
    mi = 1.0 - (float(maturity) * 0.9)
    return float(np.clip(mi, 0.2, 0.9))

# =============================
# Tree cap auto control
# =============================
def auto_limit_candidates(step_candidates: dict[str, pd.DataFrame], steps: list[str], tree_cap: int) -> dict[str, pd.DataFrame]:
    """
    If product size exceeds tree_cap, reduce each step candidates by trimming from the end (lowest Pi).
    Strategy: iteratively shrink the step with largest candidate size.
    """
    def prod_size():
        p = 1
        for s in steps:
            p *= max(1, len(step_candidates[s]))
        return p

    # Ensure min 1
    for s in steps:
        if len(step_candidates[s]) == 0:
            step_candidates[s] = step_candidates[s].head(1)

    while prod_size() > tree_cap:
        # choose step with biggest list (and >1)
        sizes = {s: len(step_candidates[s]) for s in steps}
        s_max = max(sizes, key=sizes.get)
        if sizes[s_max] <= 1:
            break
        # drop last row (lowest Pi since sorted desc)
        step_candidates[s_max] = step_candidates[s_max].iloc[:-1].copy()

    return step_candidates

# =============================
# Main
# =============================
def main():
    print("=== AASG + SPRV（軽量・安定版）: 日本語UI / Pro mode / Multi-tactic / Asset選択 / Mi ===")

    # Dictionary
    dict_path = _clean_input_path(
        input(f"技術辞書CSVパス（Enterで既定） (default: {DEFAULT_DICT_CSV}): "),
        DEFAULT_DICT_CSV
    )
    tid2name = load_dictionary(dict_path)
    print(f"辞書ロード: {len(tid2name)} 件")

    # Sector + assets
    sector = ask_choice_jp("セクター選択 (LG=自治体 / BoE=教育 / Health=医療)", ["LG", "BoE", "Health"], default="LG")
    if sector == "LG":
        assets = load_asset_list_from_txt(DEFAULT_LG_ASSET_TXT)
    elif sector == "BoE":
        assets = load_asset_list_from_txt(DEFAULT_BOE_ASSET_TXT)
    else:
        assets = load_asset_list_from_txt(DEFAULT_HEALTH_ASSET_TXT)
    print(f"資産リスト: {len(assets)} 件")

    # Load CSV
    freq_path = _clean_input_path(input(f"Frequency CSVパス（Enterで既定） (default: {DEFAULT_FREQ_CSV}): "), DEFAULT_FREQ_CSV)
    df = load_frequency(freq_path)
    print(f"入力データ: {len(df)} 行 / Unique TIDs={df['Technique_ID'].nunique()} / Unique tactics={df['Tactic_ID'].nunique()}")

    # Filters (Japanese)
    print("\n--- フィルタ設定 ---")
    do_no_mit = ask_yn_jp("Mitigation未定義のTIDのみを対象にしますか？（緩和策が定義されていないTechniqueのみ残す）", default_yes=False)
    do_no_pcd = ask_yn_jp("Persistence / Collection / Discovery を除外しますか？", default_yes=False)
    drop_no_det = ask_yn_jp("Detectionデータが無いTIDを除外しますか？（DS_Total=0を除外）", default_yes=False)

    if do_no_mit:
        df = df[pd.to_numeric(df["No_Mitigation_Flag"], errors="coerce").fillna(0).astype(int) == 1].copy()
    if do_no_pcd:
        df = df[pd.to_numeric(df["No_PCD_Flag"], errors="coerce").fillna(1).astype(int) == 1].copy()
    if drop_no_det:
        df = df[df["DS_Total"] > 0].copy()
    print(f"フィルタ後: {len(df)} 行 / Unique TIDs={df['Technique_ID'].nunique()}")

    # Steps
    step_count = ask_int("攻撃ステップ数", 1, 10, default=4)
    steps = [f"S{i}" for i in range(1, step_count + 1)]

    # Pro mode?
    pro_mode = ask_yn_jp("Pro mode（Stepごとに任意TIDを直指定）を使いますか？", default_yes=True)

    # Choose tactics per step (multi-select)
    tactics = sorted(df["Tactic_ID"].dropna().astype(str).unique().tolist())
    print("\n--- Tactic候補（全件表示）---")
    for i, t in enumerate(tactics, start=1):
        print(f"{i}) {t}")
    print("入力例: 1,4,7  / back / restart / quit")

    chosen_tactics: dict[str, list[str]] = {}
    idx = 0
    while idx < len(steps):
        s = steps[idx]
        ans = input(f"{s} に割り当てるTactic番号（複数可）: ").strip().lower()
        if ans in ("quit", "q"):
            print("終了します。")
            return
        if ans == "restart":
            print("最初からやり直します。")
            return main()
        if ans == "back":
            if idx > 0:
                idx -= 1
            else:
                print("これ以上戻れません。")
            continue

        try:
            nums = [int(x.strip()) for x in ans.split(",") if x.strip()]
            if not nums or not all(1 <= n <= len(tactics) for n in nums):
                raise ValueError
            chosen = [tactics[n-1] for n in sorted(set(nums))]
            chosen_tactics[s] = chosen
            idx += 1
        except:
            print("入力が不正です。例: 1,4,7 または back/restart/quit")

    # Candidate selection per step
    print("\n--- Step毎のTechnique候補 ---")
    step_candidates: dict[str, pd.DataFrame] = {}

    # Default TopK per step (if not Pro direct pick)
    default_topk = ask_int("候補表示TopK（上位）", 1, 200, default=20)

    for s in steps:
        tacs = chosen_tactics[s]
        cand_all = df[df["Tactic_ID"].astype(str).isin([str(x) for x in tacs])].copy()
        cand_all["Pi_num"] = pd.to_numeric(cand_all["Pi"], errors="coerce").fillna(0.0)
        # aggregate per Technique_ID: max Pi, max DS_Total, any Sector_Add
        agg = (cand_all.groupby("Technique_ID", as_index=False)
                      .agg(Pi=("Pi_num", "max"),
                           DS_Total=("DS_Total", "max"),
                           Sector_Add=("Sector_Add", "max")))
        agg = agg.sort_values("Pi", ascending=False).reset_index(drop=True)

        print(f"\n[{s}] tactic={tacs} 上位{min(default_topk, len(agg))}件")
        view = agg.head(default_topk).copy()
        for i, r in enumerate(view.itertuples(index=False), start=1):
            tid = r.Technique_ID
            name = tid2name.get(tid, "")
            nm = f" / {name}" if name else ""
            print(f"{i}) {tid}{nm}  Pi={r.Pi:.3f}  DS_Total={int(r.DS_Total)}  Sector_Add={int(r.Sector_Add)}")

        if pro_mode:
            print("Pro mode: このStepのTIDを直指定（複数可）")
            print("例: T1566.001 T1190  /  or  T1566.001,T1190")
            print("コマンド: top k / all / none / back / restart / quit")
            while True:
                cmd = input(f"{s} 入力: ").strip().lower()
                if cmd in ("quit", "q"):
                    print("終了します。")
                    return
                if cmd == "restart":
                    return main()
                if cmd == "back":
                    # go back to previous step selection
                    # easiest: re-run whole candidate selection? keep simple: allow "none" here and user can restart.
                    print("この画面だけbackは複雑になるため、restart を推奨します。")
                    continue
                if cmd.startswith("top"):
                    m = re.match(r"^top\s+(\d+)$", cmd)
                    if not m:
                        print("例: top 4")
                        continue
                    k = int(m.group(1))
                    k = max(1, min(k, len(agg)))
                    step_candidates[s] = agg.head(k).copy()
                    break
                if cmd == "all":
                    step_candidates[s] = agg.copy()
                    break
                if cmd in ("none", ""):
                    print("none は不可（直積が作れなくなるため）。最低1つ選んでください。")
                    continue

                # treat as TID list
                tids = parse_tid_list(cmd)
                if not tids:
                    print("TIDが解析できません。例: T1078.004 T1190")
                    continue

                # IMPORTANT: match against full agg (not only top list)
                picked = agg[agg["Technique_ID"].isin(tids)].copy()

                # if some tids not found, tell which
                missing = [t for t in tids if t not in set(agg["Technique_ID"].tolist())]
                if missing:
                    print(f"指定TIDの一部が見つかりません: {missing}")
                    print("→ tacticフィルタで落ちている可能性があります。tactic選択を見直すか、top/allを使ってください。")
                if picked.empty:
                    print("指定TIDが見つかりません（このStepのtactic条件に合致しない）。")
                    continue

                # keep order as input
                picked["__ord"] = picked["Technique_ID"].apply(lambda x: tids.index(x) if x in tids else 999999)
                picked = picked.sort_values("__ord").drop(columns="__ord")
                step_candidates[s] = picked.copy()
                print(f"{s}: 選択TID数 = {len(step_candidates[s])}")
                break
        else:
            # non-pro mode: choose topk
            k = ask_int(f"{s} のTopK（候補数）", 1, 200, default=min(4, len(agg)) if len(agg) else 1)
            step_candidates[s] = agg.head(k).copy()

    # Detection Nested Practice Mode
    detection_all = ask_yn_jp("Nested Practice Mode: 全TIDを detection=all として一括登録しますか？", default_yes=True)

    # Asset selection per step (1 asset per step) with back/restart
    print("\n--- Asset割当（Step毎に1つ選択）---")
    for i, a in enumerate(assets, start=1):
        print(f"{i}) {a}")
    print("入力例: 3  / manual / back / restart / quit")

    step_asset: dict[str, str] = {}
    i = 0
    while i < len(steps):
        s = steps[i]
        ans = input(f"{s} のAsset番号: ").strip().lower()
        if ans in ("quit", "q"):
            print("終了します。")
            return
        if ans == "restart":
            return main()
        if ans == "back":
            if i > 0:
                i -= 1
            else:
                print("これ以上戻れません。")
            continue
        if ans in ("manual", "m"):
            txt = input("Asset名を手入力: ").strip()
            if not txt:
                print("空は不可です。")
                continue
            step_asset[s] = txt
            i += 1
            continue
        if ans.isdigit():
            n = int(ans)
            if 1 <= n <= len(assets):
                step_asset[s] = assets[n-1]
                i += 1
                continue
        print("入力が不正です。番号、manual、back、restart、quit")

    # Tree cap auto-control
    tree_cap = ask_int("攻撃ツリー上限（自動制御）", 1, 20000, default=2000)
    # sort candidates by Pi desc already; ensure DF not empty
    for s in steps:
        if len(step_candidates[s]) == 0:
            step_candidates[s] = step_candidates[s].head(1)

    # compute product and auto shrink if needed
    raw_prod = 1
    for s in steps:
        raw_prod *= len(step_candidates[s])
    print(f"\n直積の攻撃ツリー数(縮小前): {raw_prod}")
    if raw_prod > tree_cap:
        print(f"上限 {tree_cap} を超えるため、自動で候補数を削減します（低Piから落とす）。")
        step_candidates = auto_limit_candidates(step_candidates, steps, tree_cap)

    prod = 1
    for s in steps:
        prod *= len(step_candidates[s])
    print(f"攻撃ツリー数(縮小後): {prod}")
    for s in steps:
        print(f" {s}: candidates={len(step_candidates[s])}")

    # Build trees (cartesian)
    lists = [step_candidates[s]["Technique_ID"].astype(str).tolist() for s in steps]
    trees = list(itertools.product(*lists))

    # Build output LONG
    out_rows = []
    for tree_id, tids in enumerate(trees, start=1):
        r_prev = 0.0
        for step_i, (s, tid) in enumerate(zip(steps, tids), start=1):
            tid = normalize_tid(tid)

            # retrieve info from df: use ANY row matching this tid (ignoring tactic in this step) to get DS list
            # but for consistency, use aggregated table we selected:
            info = step_candidates[s][step_candidates[s]["Technique_ID"] == tid].head(1)
            pi = float(info["Pi"].iloc[0]) if len(info) else 0.0
            ds_total = int(info["DS_Total"].iloc[0]) if len(info) else 0
            sector_add = int(info["Sector_Add"].iloc[0]) if len(info) else 0

            # maturity / Mi
            if detection_all:
                ds_sel = ds_total
                maturity = 1.0 if ds_total > 0 else 0.0
                mi = mi_from_maturity(maturity)
            else:
                # if not all, ask ratio only (lightweight)
                if ds_total <= 0:
                    ds_sel = 0
                    maturity = 0.0
                    mi = mi_from_maturity(maturity)
                else:
                    ds_sel = ask_int(f"[Tree {tree_id}] {s} {tid} 検知できている数(0-{ds_total})", 0, ds_total, default=ds_total)
                    maturity = ds_sel / ds_total if ds_total else 0.0
                    mi = mi_from_maturity(maturity)

            base_risk = pi * mi
            propag = (1.0 + r_prev)
            ri = base_risk * propag
            r_prev = ri

            out_rows.append({
                "Scenario_ID": f"{sector}-S1",
                "AttackTree_ID": tree_id,
                "Step": s,
                "Step_num": step_i,
                "Technique_ID": tid,
                "Technique_Name_J": tid2name.get(tid, ""),
                "Asset": step_asset.get(s, ""),
                "Pi": pi,
                "DS_Total": ds_total,
                "DS_Selected": ds_sel,
                "Maturity": maturity,
                "Mi": mi,
                "Base_Risk": base_risk,
                "R_prev": (propag - 1.0),
                "Propagation_Term": propag,
                "Ri": ri,
                "Sector_Add": sector_add
            })

    out_long = pd.DataFrame(out_rows)

    # R_total per tree (Ri of last step)
    out_long = out_long.sort_values(["AttackTree_ID", "Step_num"])
    rtotal = (out_long.groupby(["Scenario_ID", "AttackTree_ID"], as_index=False)
                     .tail(1)[["Scenario_ID", "AttackTree_ID", "Ri"]]
                     .rename(columns={"Ri": "R_total"}))
    out_long = out_long.merge(rtotal, on=["Scenario_ID", "AttackTree_ID"], how="left")

    # Save
    out_dir = "/mnt/data"
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    out_path = f"{out_dir}/AASG_SPRV_FREQUENCY_{sector}_LONG.csv"
    out_long.to_csv(out_path, index=False, encoding="utf-8-sig")

    print("\n=== DONE ===")
    print("Saved:", out_path)
    print("Rows:", len(out_long))
    print("Mi min/max:", float(out_long["Mi"].min()), float(out_long["Mi"].max()))
    print("Sector_Add sum:", int(out_long["Sector_Add"].sum()))

if __name__ == "__main__":
    main()


=== AASG + SPRV（軽量・安定版）: 日本語UI / Pro mode / Multi-tactic / Asset選択 / Mi ===
技術辞書CSVパス（Enterで既定） (default: /mnt/data/arks_dictionary_J_ver3_phase0_asset_independent.csv): 
辞書ロード: 823 件
セクター選択 (LG=自治体 / BoE=教育 / Health=医療) (default LG): LG
資産リスト: 20 件
Frequency CSVパス（Enterで既定） (default: /mnt/data/ARKS_Task1_with_Pi_and_DataSources.csv):  /mnt/data/ARKS_Task1_with_Pi_and_DataSources.csv
入力データ: 874 行 / Unique TIDs=679 / Unique tactics=14

--- フィルタ設定 ---
Mitigation未定義のTIDのみを対象にしますか？（緩和策が定義されていないTechniqueのみ残す） [y/n] (default n): n
Persistence / Collection / Discovery を除外しますか？ [y/n] (default n): n
Detectionデータが無いTIDを除外しますか？（DS_Total=0を除外） [y/n] (default n): n
フィルタ後: 874 行 / Unique TIDs=679
攻撃ステップ数 [1-10] (default 4): 4
Pro mode（Stepごとに任意TIDを直指定）を使いますか？ [y/n] (default y): y

--- Tactic候補（全件表示）---
1) collection
2) command-and-control
3) credential-access
4) defense-evasion
5) discovery
6) execution
7) exfiltration
8) impact
9) initial-access
10) lateral-movement
11) persistence
12) privilege-es